In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.svm import OneClassSVM

# Importing and combining the datasets
df1 = pd.read_csv('../Datasets/NB.csv')
df1['Status'] = 0
df2 = pd.read_csv('../Datasets/IR - 7.csv')
df2['Status'] = 1
train_data = pd.concat([df1[['DE', 'FE', 'Status']], df2[['DE', 'FE', 'Status']]], ignore_index=True)

# Splitting the data into training and testing sets
df1_train, df1_test = train_test_split(df1, test_size=0.2, shuffle=False, random_state=0)
df2_train, df2_test = train_test_split(df2, test_size=0.2, shuffle=False, random_state=0)

# Scaling the data
scaler = StandardScaler()
train_data_scaled = scaler.fit_transform(train_data)
test_data_scaled = scaler.transform(pd.concat([df1_test[['DE', 'FE', 'Status']], df2_test[['DE', 'FE', 'Status']]], ignore_index=True))

# Training the OneClassSVM model
outliers_fraction = 0.01
model = OneClassSVM(nu=outliers_fraction, kernel="rbf", gamma=0.01)
pred_train = model.fit(train_data_scaled)

# Predicting anomalies in the training and testing sets
train_data['Anomaly'] = pd.Series(model.predict(train_data_scaled))
test_data = pd.concat([df1_test[['DE', 'FE', 'Status']], df2_test[['DE', 'FE', 'Status']]], ignore_index=True)
test_data['Anomaly'] = pd.Series(model.predict(test_data_scaled))

# Visualizing the anomalies
train_anomalies = train_data[train_data['Anomaly'] == -1]
test_anomalies = test_data[test_data['Anomaly'] == -1]

f, (ax1, ax2) = plt.subplots(2, 1, figsize=(18, 12))
ax1.scatter(train_anomalies.index, train_anomalies.DE, label='Anomaly', color='red', s=10)
ax1.plot(train_data.index, train_data.DE, label='DE')
ax1.set_xlim((0, len(train_data.index)))
ax1.set_title('One Class SVM (Training Set)')
ax1.legend()

ax2.scatter(test_anomalies.index, test_anomalies.DE, label='Anomaly', color='red', s=10)
ax2.plot(test_data.index, test_data.DE, label='DE')
ax2.set_xlim((0, len(test_data.index)))
ax2.set_title('One Class SVM (Testing Set)')
ax2.legend()

plt.show()

# Computing accuracy and number of anomalies found in the testing set
score = 100 * accuracy_score(test_data['Anomaly'], test_data['Status'])
num_anomalies = test_anomalies['Anomaly'].count()
print("Accuracy: {:.2f}%".format(score))
print("Anomalies found: {}".format(num_anomalies))
